In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import time

import cv2
from torch.utils.data import DataLoader, Dataset
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torchvision.transforms as transforms


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score

import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [14]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    r"""
    3x3 convolution with padding
    - in_planes: in_channels
    - out_channels: out_channels
    - bias=False: BatchNorm에 bias가 포함되어 있으므로, conv2d는 bias=False로 설정.
    """
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [15]:
class CustomDataset(Dataset):
    def __init__(self, image_path_list, label_list):
        self.image_path_list = image_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        
        path = self.image_path_list[index]

        frames = self.get_image(path)
            
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.image_path_list)
    
    def get_image(self, path):
        # Read image
        img = cv2.imread(path)
        if img is None:
            print(f"Error reading image file {path}")
            return torch.empty(0) 
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ''' height, width = img.shape[:2]
        
        if CFG['CROP_SIZE'] > 0 and (width - CFG['CROP_SIZE'] * 2) > 0:
            img = img[:, CFG['CROP_SIZE']:width - CFG['CROP_SIZE']]
        else:
            print(f"Invalid crop size for image {path}")
            return torch.empty(0)
        img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        img = img / 255.0  # Normalize to [0, 1]     '''      
            
        return torch.FloatTensor(img).permute(2, 0, 1)

In [16]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        r"""
         - inplanes: input channel size
         - planes: output channel size
         - groups, base_width: ResNext나 Wide ResNet의 경우 사용
        """
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
            
        # Basic Block의 구조
        self.conv1 = conv3x3(inplanes, planes, stride)  # conv1에서 downsample
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        # short connection
        if self.downsample is not None:
            identity = self.downsample(x)
            
        # identity mapping시 identity mapping후 ReLU를 적용합니다.
        # 그 이유는, ReLU를 통과하면 양의 값만 남기 때문에 Residual의 의미가 제대로 유지되지 않기 때문입니다.
        out += identity
        out = self.relu(out)

        return out

In [17]:
class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4 # 블록 내에서 차원을 증가시키는 3번째 conv layer에서의 확장계수
    
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        # ResNext나 WideResNet의 경우 사용
        width = int(planes * (base_width / 64.)) * groups
        
        # Bottleneck Block의 구조
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation) # conv2에서 downsample
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        # 1x1 convolution layer
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        # 3x3 convolution layer
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        # 1x1 convolution layer
        out = self.conv3(out)
        out = self.bn3(out)
        # skip connection
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [18]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        # default values
        self.inplanes = 64 # input feature map
        self.dilation = 1
        # stride를 dilation으로 대체할지 선택
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        
        r"""
        - 처음 입력에 적용되는 self.conv1과 self.bn1, self.relu는 모든 ResNet에서 동일 
        - 3: 입력으로 RGB 이미지를 사용하기 때문에 convolution layer에 들어오는 input의 channel 수는 3
        """
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        r"""
        - 아래부터 block 형태와 갯수가 ResNet층마다 변화
        - self.layer1 ~ 4: 필터의 개수는 각 block들을 거치면서 증가(64->128->256->512)
        - self.avgpool: 모든 block을 거친 후에는 Adaptive AvgPool2d를 적용하여 (n, 512, 1, 1)의 텐서로
        - self.fc: 이후 fc layer를 연결
        """
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, # 여기서부터 downsampling적용
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        r"""
        convolution layer 생성 함수
        - block: block종류 지정
        - planes: feature map size (input shape)
        - blocks: layers[0]와 같이, 해당 블록이 몇개 생성돼야하는지, 블록의 갯수 (layer 반복해서 쌓는 개수)
        - stride와 dilate은 고정
        """
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        
        # the number of filters is doubled: self.inplanes와 planes 사이즈를 맞춰주기 위한 projection shortcut
        # the feature map size is halved: stride=2로 downsampling
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        # 블록 내 시작 layer, downsampling 필요
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion # inplanes 업데이트
        # 동일 블록 반복
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

In [19]:
def train_model(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = F.cross_entropy
    
    train_losses = []
    val_losses = []
    val_scores = []
    
    best_val_score = 0
    best_model = None
    
    count = 1

    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        
        path = 'weight/1002_' + str(count) + '.pth'
        
        for images, labels in tqdm(iter(train_loader)):
            
            images = images.to(device)
            labels = labels.to(device)
            #print("real : ", labels)
            
            optimizer.zero_grad()
            
            output = model(images)
            
            main_loss = criterion(output.float(), labels.long())
            
            total_loss = main_loss
            
            total_loss.backward()
            optimizer.step()
            
            train_loss.append(total_loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        train_losses.append(_train_loss)
        val_losses.append(_val_loss)
        val_scores.append(_val_score)
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
            

        # torch.save(model.state_dict(), path)
        count = count + 1
            
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train and Validation Losses')
    plt.legend()
    
    #plt.savefig('./4. Loss/0320_epoch20.png')
    
    plt.show()
    
    return best_model

In [20]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit.float(), labels.long())
            #print(loss)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [21]:
def inference(model, test_loader, device):
    count = 1
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for images in tqdm(iter(test_loader)):
            images = images.to(device)
            logit = model(images)
            #print(count)
            #print(logit)
            #print()
            count = count + 1
            #print("logit: ",logit)
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            #print("preds: ",preds)

            #print()
            
            
    return preds

In [22]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41,
    'CROP_SIZE':200
}

# Train

In [23]:
def train_n_infer(case_num):
    case_name = f"Case_{case_num}"
    print(f"{case_name} 처리 중...")

    # CSV 파일 불러오기
    train = pd.read_csv(f"/home/wkdtjdqls/jsb_ws/network/07. cnn/re/Protocol_Data/{case_name}/TRAIN.csv")
    val = pd.read_csv(f"/home/wkdtjdqls/jsb_ws/network/07. cnn/re/Protocol_Data/{case_name}/VAL.csv")
    test = pd.read_csv(f"/home/wkdtjdqls/jsb_ws/network/07. cnn/re/Protocol_Data/{case_name}/TEST.csv")

    # 데이터 로더 설정
    train_dataset = CustomDataset(train['path'].values, train['label'].values)
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
    val_dataset = CustomDataset(val['path'].values, val['label'].values)
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
    test_dataset = CustomDataset(test['path'].values, None)
    test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

    # 모델 및 최적화 설정
    model = ResNet(block=BasicBlock, layers=[2, 2, 2, 2]).to(device)
    optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)


    # 모델 학습
    infer_model = train_model(model, optimizer, train_loader, val_loader, scheduler, device)

    # 학습된 모델 저장
    torch.save(infer_model.state_dict(), f"/home/wkdtjdqls/jsb_ws/network/07. cnn/re/weight/resnet/{case_name}_model.pth")
    print(f"{case_name} 학습 완료 및 모델 저장")

In [24]:
for i in range(1, 31):
    train_n_infer(i)

Case_1 처리 중...


/home/wkdtjdqls/anaconda3/envs/jsb/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


  0%|          | 0/18435 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Prediction

In [25]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score
from torch.utils.data import DataLoader

# 결과 저장용 리스트 초기화
results = []

def test_n_infer(case_num):
    case_name = f"Case_{case_num}"
    print(f"{case_name} 처리 중...")
    
    model = ResNet(block=BasicBlock, layers=[2, 2, 2, 2])
    weight_path = f"/home/wkdtjdqls/jsb_ws/network/07. cnn/re/weight/Case_{case_num}_model.pth"
    model.load_state_dict(torch.load(weight_path))
    
    # CSV 파일 불러오기
    test = pd.read_csv(f"/home/wkdtjdqls/jsb_ws/network/07. cnn/re/Protocol_Data/{case_name}/TEST.csv")

    # 데이터 로더 설정
    test_dataset = CustomDataset(test['path'].values, None)
    test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

    # 모델 추론
    preds = inference(model, test_loader, device)

    # 테스트 레이블과 비교
    test_result = test['label']

    # 메트릭 계산
    score_acc = accuracy_score(test_result, preds)
    score_f1 = f1_score(test_result, preds, average='weighted')
    score_precision = precision_score(test_result, preds, average='weighted')

    print(f"Accuracy: {score_acc:.4f}, F1 Score: {score_f1:.4f}, Precision: {score_precision:.4f}")

    # 결과 저장
    results.append({
        'Case': case_name,
        'Accuracy': score_acc,
        'F1_Score': score_f1,
        'Precision': score_precision
    })

In [26]:
# case1부터 case30까지 실행
for i in range(1, 31):
    test_n_infer(i)

Case_1 처리 중...


/tmp/ipykernel_64615/3855773929.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path))


  0%|          | 0/18879 [00:00<?, ?it/s]

Accuracy: 0.9366, F1 Score: 0.9337, Precision: 0.9373
Case_2 처리 중...


/tmp/ipykernel_64615/3855773929.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path))


...

  0%|          | 0/18879 [00:00<?, ?it/s]

Accuracy: 0.9394, F1 Score: 0.9373, Precision: 0.9431
Case_30 처리 중...


/tmp/ipykernel_64615/3855773929.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path))


  0%|          | 0/18879 [00:00<?, ?it/s]

Accuracy: 0.9118, F1 Score: 0.9001, Precision: 0.9186


In [30]:
df_results = pd.DataFrame(results)
df_results.to_csv("/home/wkdtjdqls/jsb_ws/network/07. cnn/re/result/case_acc_f1_prec_results.csv", index=False)

print("CSV 파일로 저장 완료!")

CSV 파일로 저장 완료!


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader


# case 결과 저장용 리스트 초기화
results = []
# 모든 케이스의 프로토콜별 결과를 저장할 리스트 초기화
all_case_results = []

def test_and_visualize(case_num):
    case_name = f"Case_{case_num}"
    print(f"{case_name} 처리 중...")

    # 모델 초기화 및 가중치 로드
    model = ResNet(block=BasicBlock, layers=[2, 2, 2, 2])
    weight_path = f"/home/wkdtjdqls/jsb_ws/network/07. cnn/re/weight/Case_{case_num}_model.pth"
    model.load_state_dict(torch.load(weight_path))

    # CSV 파일 불러오기
    test_data = pd.read_csv(f"/home/wkdtjdqls/jsb_ws/network/07. cnn/re/Protocol_Data/{case_name}/TEST.csv")

    # 데이터 로더 설정
    test_dataset = CustomDataset(test_data['path'].values, None)
    test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

    # 모델 추론
    preds = inference(model, test_loader, device)
    
    # 테스트 레이블과 비교
    test_result = test_data['label']

    # 메트릭 계산
    score_acc = accuracy_score(test_result, preds)
    score_f1 = f1_score(test_result, preds, average='weighted')
    score_precision = precision_score(test_result, preds, average='weighted')

    print(f"Accuracy: {score_acc:.4f}, F1 Score: {score_f1:.4f}, Precision: {score_precision:.4f}")

    # case마다의 결과 저장
    results.append({
        'Case': case_name,
        'Accuracy': score_acc,
        'F1_Score': score_f1,
        'Precision': score_precision
    })

    # 테스트 결과 비교
    preds = np.array(preds)
    classes = test_data['label'].unique()
    class_accuracy = {}

    # 각 클래스에 대해 정확도 계산
    for class_label in classes:
        class_mask = test_data['label'] == class_label
        class_preds = preds[class_mask]
        class_result = test_data['label'][class_mask]
        accuracy = accuracy_score(class_result, class_preds)
        class_accuracy[class_label] = accuracy

    # 결과 저장 형식: {Case, Class, Accuracy}
    for label, accuracy in class_accuracy.items():
        all_case_results.append({
            'Case': case_name,
            'Class': label,
            'Accuracy': accuracy
        })

# Case 1부터 30까지 실행
for i in range(1, 31):
    test_and_visualize(i)
    
df_results = pd.DataFrame(results)
df_results.to_csv("/home/wkdtjdqls/jsb_ws/network/07. cnn/re/result/case_acc_f1_prec_results.csv", index=False)

# 전체 결과를 DataFrame으로 변환 후 CSV로 저장
df_all_results = pd.DataFrame(all_case_results)
df_all_results.to_csv("/home/wkdtjdqls/jsb_ws/network/07. cnn/re/result/class_accuracy_results.csv", index=False)

print("완료!")


Case_1 처리 중...


/tmp/ipykernel_64615/2872137909.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path))


  0%|          | 0/18879 [00:00<?, ?it/s]

Accuracy: 0.9366, F1 Score: 0.9337, Precision: 0.9373
Case_2 처리 중...


/tmp/ipykernel_64615/2872137909.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path))


...

  0%|          | 0/18879 [00:00<?, ?it/s]

Accuracy: 0.9394, F1 Score: 0.9373, Precision: 0.9431
Case_30 처리 중...


/tmp/ipykernel_64615/2872137909.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path))


  0%|          | 0/18879 [00:00<?, ?it/s]

Accuracy: 0.9118, F1 Score: 0.9001, Precision: 0.9186
완료!


In [ ]:
'''train = pd.read_csv('/media/wkdtjdqls/easymean_1/Protocol_Data/Case_1/TRAIN.csv')
val = pd.read_csv('/media/wkdtjdqls/easymean_1/Protocol_Data/Case_1/VAL.csv')
test = pd.read_csv('/media/wkdtjdqls/easymean_1/Protocol_Data/Case_1/TEST.csv')


train_dataset = CustomDataset(train['image_path'].values, train['class'].values)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
val_dataset = CustomDataset(val['image_path'].values, val['class'].values)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
test_dataset = CustomDataset(test_123['image_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


model = ResNet(block=BasicBlock, layers=[2, 2, 2, 2])
#model.load_state_dict(torch.load("./2. Weight/0320_3.pth"))
#model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

%%time
preds = inference(model, test_loader, device)'''

In [ ]:
'''model = ResNet(block=BasicBlock, layers=[2, 2, 2, 2])
#model.load_state_dict(torch.load("./2. Weight/0320_3.pth"))
#model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)'''